# <center>TITRE - Projet Python</center>

## Membres:
* Corentin Drezen - 40007337, TD2
* Nathanael NOM - N°ETUDIANT, TD1
* Irfane NOM - N°ETUDIANT, TD2

## Introduction:
On souhaite réaliser un jeu du sytle <i>shoot them up</i> en 2D à l'aide du module $\color{#B267E6}{\text{pygame}}$ avec le personnage principal qui suit la souris

<img src="SpaceInvaders-Gameplay.gif">
